In [148]:
import nltk
import pandas as pd
import numpy as np
import random

In [149]:
netflix_df = pd.read_csv(r'C:\Users\ahmad\Desktop\sap\adm\HW4\archive (1)\vodclickstream_uk_movies_03.csv')

print(netflix_df.describe())
print(netflix_df.columns)

          Unnamed: 0      duration
count  671736.000000  6.717360e+05
mean   394640.500000  3.347603e+04
std    193913.624549  1.574396e+05
min     58773.000000 -1.000000e+00
25%    226706.750000  0.000000e+00
50%    394640.500000  1.400000e+01
75%    562574.250000  6.672250e+03
max    730508.000000  1.823725e+07
Index(['Unnamed: 0', 'datetime', 'duration', 'title', 'genres', 'release_date',
       'movie_id', 'user_id'],
      dtype='object')


In [150]:
# a copy of the dataframe so i dont overwrite
succ_movies = netflix_df.copy()
succ_movies.drop(['movie_id', 'release_date','datetime'], axis = 1)
succ_movies = succ_movies.sort_values(["user_id", "movie_id", "datetime", "duration"])
succ_movies

,Unnamed: 0,datetime,duration,title,genres,release_date,movie_id,user_id
223802,282575,2017-12-05 20:39:15,0.0,Hannibal,"Crime, Drama, Thriller",2001-02-09,9bfee795ff,00004e2862
89781,148554,2017-05-30 20:36:34,0.0,Green Room,"Horror, Music, Thriller",2016-05-13,43bb6de314,000052a0a0
102939,161712,2017-06-18 20:38:02,0.0,Looper,"Action, Drama, Sci-Fi, Thriller",2012-09-28,4718f9963c,000052a0a0
102994,161767,2017-06-18 20:44:27,0.0,Looper,"Action, Drama, Sci-Fi, Thriller",2012-09-28,4718f9963c,000052a0a0
103016,161789,2017-06-18 20:44:27,0.0,Looper,"Action, Drama, Sci-Fi, Thriller",2012-09-28,4718f9963c,000052a0a0
...,...,...,...,...,...,...,...,...
143135,201908,2017-08-11 21:03:15,2171.0,Amanda Knox,"Documentary, Crime",2016-09-30,cb6d7afd76,fffeac83be
142728,201501,2017-08-10 07:15:41,49440.0,Enemy at the Gates,"Drama, History, War",2001-03-16,dda0eae17b,fffeac83be
130095,188868,2017-07-23 15:02:43,0.0,Hot Fuzz,"Action, Comedy, Mystery, Thriller",2007-04-20,6467fee6b6,ffff2c5f9e
129867,188640,2017-07-23 14:45:12,0.0,Forks Over Knives,Documentary,2013-10-16,9ab62a3f2c,ffff2c5f9e


In [151]:
# calculating number of clicks
#RQ 1,1
user_movie_clicks = succ_movies.groupby(['user_id', 'movie_id', 'title','genres']).size().reset_index(name='clicks')

user_movie_clicks

,user_id,movie_id,title,genres,clicks
0,00004e2862,9bfee795ff,Hannibal,"Crime, Drama, Thriller",1
1,000052a0a0,43bb6de314,Green Room,"Horror, Music, Thriller",1
2,000052a0a0,4718f9963c,Looper,"Action, Drama, Sci-Fi, Thriller",9
3,000052a0a0,4fa0b092d6,Jumanji,"Adventure, Comedy, Family, Fantasy",3
4,000052a0a0,59d313ed8b,Ant-Man,"Action, Adventure, Comedy, Sci-Fi",1
...,...,...,...,...,...
502502,fffeac83be,cb6d7afd76,Amanda Knox,"Documentary, Crime",2
502503,fffeac83be,dda0eae17b,Enemy at the Gates,"Drama, History, War",1
502504,ffff2c5f9e,6467fee6b6,Hot Fuzz,"Action, Comedy, Mystery, Thriller",1
502505,ffff2c5f9e,9ab62a3f2c,Forks Over Knives,Documentary,1


In [41]:
#Filtering the top movies per user for genres column, to perform hashing
import re
top_movies_per_user = user_movie_clicks.sort_values('clicks', ascending=False).groupby('user_id').head(10)

top_movies_per_user.sort_values(by = ['user_id','clicks'], ascending = False)
top_movies_per_user['filtered_genres'] = top_movies_per_user['genres'].apply(lambda x: re.sub(r'[^\w\s]', '', x.lower()))

top_movies_per_user = top_movies_per_user.drop_duplicates(subset = 'user_id')
top_movies_per_user

,user_id,movie_id,title,genres,clicks,filtered_genres
245742,7cdfd0e14a,40bccd3001,Twilight,"Drama, Fantasy, Romance",88,drama fantasy romance
69019,23c52f9b50,df00041e47,Monty Python and the Holy Grail,"Adventure, Comedy, Fantasy",62,adventure comedy fantasy
175415,59416738c3,cbdf9820bc,The Ex,"Comedy, Romance",54,comedy romance
440212,e06f0be797,3f3b34e56f,Rush Hour 3,"Action, Comedy, Crime, Thriller",54,action comedy crime thriller
144718,49d091aa63,b8a2658c23,Chalet Girl,"Comedy, Romance, Sport",48,comedy romance sport
...,...,...,...,...,...,...
187813,5fdd93fc0c,fba9fa3a5a,The Beach,"Adventure, Drama, Romance, Thriller",1,adventure drama romance thriller
187812,5fdd462bd5,8c14e58ed6,Safe Haven,"Drama, Romance, Thriller",1,drama romance thriller
187811,5fdc850e43,9a94a918d1,Cinderella,"Drama, Family, Fantasy, Romance",1,drama family fantasy romance
187807,5fdc7f6d2f,6f8ac8a090,Carrie,"Drama, Horror, Sci-Fi, Thriller",1,drama horror scifi thriller


In [77]:
# Shingle creation of genres column for hashing
def create_shingle(shingle_size, texts):
    
    shingles_list = []

  
    for text in texts:
        shingles = [text[i:i + shingle_size] for i in range(len(text) - shingle_size + 1)]
        shingles_list.append(shingles)
    
    return shingles_list




[['spo', 'por', 'ort'], ['his', 'ist', 'sto', 'tor', 'ory'], ['mus', 'usi', 'sic', 'ica', 'cal'], ['ava', 'vai', 'ail', 'ila', 'lab', 'abl', 'ble'], ['new', 'ews'], ['com', 'ome', 'med', 'edy'], ['mys', 'yst', 'ste', 'ter', 'ery'], ['sci', 'cif', 'ifi'], ['sho', 'hor', 'ort'], ['doc', 'ocu', 'cum', 'ume', 'men', 'ent', 'nta', 'tar', 'ary'], ['bio', 'iog', 'ogr', 'gra', 'rap', 'aph', 'phy'], ['cri', 'rim', 'ime'], ['fan', 'ant', 'nta', 'tas', 'asy'], ['mus', 'usi', 'sic'], ['dra', 'ram', 'ama'], ['hor', 'orr', 'rro', 'ror'], ['fil', 'ilm', 'lmn', 'mno', 'noi', 'oir'], ['not'], ['tal', 'alk', 'lks', 'ksh', 'sho', 'how'], ['ani', 'nim', 'ima', 'mat', 'ati', 'tio', 'ion'], ['thr', 'hri', 'ril', 'ill', 'lle', 'ler'], ['war'], ['wes', 'est', 'ste', 'ter', 'ern'], ['act', 'cti', 'tio', 'ion'], ['rea', 'eal', 'ali', 'lit', 'ity', 'tyt', 'ytv'], ['rom', 'oma', 'man', 'anc', 'nce'], ['fam', 'ami', 'mil', 'ily'], ['adv', 'dve', 'ven', 'ent', 'ntu', 'tur', 'ure']]


In [135]:

def hashing(text,a = 3.91):
    hash_val = 0
    
    for char in text:
        hash_val = round((hash_val * 3.91)) + ord(char)
    return hash_val
    

In [115]:

top_movies_per_user['shingles'] = create_shingle(shingle_size=3, texts=top_movies_per_user['filtered_genres'].tolist())

top_movies_per_user

,user_id,movie_id,title,genres,clicks,filtered_genres,shingles
245742,7cdfd0e14a,40bccd3001,Twilight,"Drama, Fantasy, Romance",88,drama fantasy romance,"[dra, ram, ama, ma , a f, fa, fan, ant, nta, ..."
69019,23c52f9b50,df00041e47,Monty Python and the Holy Grail,"Adventure, Comedy, Fantasy",62,adventure comedy fantasy,"[adv, dve, ven, ent, ntu, tur, ure, re , e c, ..."
175415,59416738c3,cbdf9820bc,The Ex,"Comedy, Romance",54,comedy romance,"[com, ome, med, edy, dy , y r, ro, rom, oma, ..."
440212,e06f0be797,3f3b34e56f,Rush Hour 3,"Action, Comedy, Crime, Thriller",54,action comedy crime thriller,"[act, cti, tio, ion, on , n c, co, com, ome, ..."
144718,49d091aa63,b8a2658c23,Chalet Girl,"Comedy, Romance, Sport",48,comedy romance sport,"[com, ome, med, edy, dy , y r, ro, rom, oma, ..."
...,...,...,...,...,...,...,...
187813,5fdd93fc0c,fba9fa3a5a,The Beach,"Adventure, Drama, Romance, Thriller",1,adventure drama romance thriller,"[adv, dve, ven, ent, ntu, tur, ure, re , e d, ..."
187812,5fdd462bd5,8c14e58ed6,Safe Haven,"Drama, Romance, Thriller",1,drama romance thriller,"[dra, ram, ama, ma , a r, ro, rom, oma, man, ..."
187811,5fdc850e43,9a94a918d1,Cinderella,"Drama, Family, Fantasy, Romance",1,drama family fantasy romance,"[dra, ram, ama, ma , a f, fa, fam, ami, mil, ..."
187807,5fdc7f6d2f,6f8ac8a090,Carrie,"Drama, Horror, Sci-Fi, Thriller",1,drama horror scifi thriller,"[dra, ram, ama, ma , a h, ho, hor, orr, rro, ..."


In [147]:


def min_hashing(shingle, num_hashes,hashing):
    np.random.seed(43) 
    hash_values = [hashing(char for char in shingle)]
    mh_sig = [min((hash_value * i + 1) % 1000000 for hash_value in hash_values) for i in range(1, num_hashes + 1)]
    return mh_sig

num_hashes = 50

top_movies_per_user['minhash_sig'] = top_movies_per_user['shingles'].apply(lambda x: min_hashing(x,num_hashes,hashing))



TypeError: ord() expected a character, but string of length 3 found

In [139]:
top_movies_per_user

,user_id,movie_id,title,genres,clicks,filtered_genres,shingles,minhash_sig
245742,7cdfd0e14a,40bccd3001,Twilight,"Drama, Fantasy, Romance",88,drama fantasy romance,"[dra, ram, ama, ma , a f, fa, fan, ant, nta, ...","[985, 1970, 2955, 3940, 4925, 5910, 6895, 7880..."
69019,23c52f9b50,df00041e47,Monty Python and the Holy Grail,"Adventure, Comedy, Fantasy",62,adventure comedy fantasy,"[adv, dve, ven, ent, ntu, tur, ure, re , e c, ...","[985, 1970, 2955, 3940, 4925, 5910, 6895, 7880..."
175415,59416738c3,cbdf9820bc,The Ex,"Comedy, Romance",54,comedy romance,"[com, ome, med, edy, dy , y r, ro, rom, oma, ...","[1045, 2090, 3135, 4180, 5225, 6270, 7315, 836..."
440212,e06f0be797,3f3b34e56f,Rush Hour 3,"Action, Comedy, Crime, Thriller",54,action comedy crime thriller,"[act, cti, tio, ion, on , n c, co, com, ome, ...","[987, 1974, 2961, 3948, 4935, 5922, 6909, 7896..."
144718,49d091aa63,b8a2658c23,Chalet Girl,"Comedy, Romance, Sport",48,comedy romance sport,"[com, ome, med, edy, dy , y r, ro, rom, oma, ...","[1045, 2090, 3135, 4180, 5225, 6270, 7315, 836..."
...,...,...,...,...,...,...,...,...
187813,5fdd93fc0c,fba9fa3a5a,The Beach,"Adventure, Drama, Romance, Thriller",1,adventure drama romance thriller,"[adv, dve, ven, ent, ntu, tur, ure, re , e d, ...","[994, 1988, 2982, 3976, 4970, 5964, 6958, 7952..."
187812,5fdd462bd5,8c14e58ed6,Safe Haven,"Drama, Romance, Thriller",1,drama romance thriller,"[dra, ram, ama, ma , a r, ro, rom, oma, man, ...","[1045, 2090, 3135, 4180, 5225, 6270, 7315, 836..."
187811,5fdc850e43,9a94a918d1,Cinderella,"Drama, Family, Fantasy, Romance",1,drama family fantasy romance,"[dra, ram, ama, ma , a f, fa, fam, ami, mil, ...","[985, 1970, 2955, 3940, 4925, 5910, 6895, 7880..."
187807,5fdc7f6d2f,6f8ac8a090,Carrie,"Drama, Horror, Sci-Fi, Thriller",1,drama horror scifi thriller,"[dra, ram, ama, ma , a h, ho, hor, orr, rro, ...","[1006, 2012, 3018, 4024, 5030, 6036, 7042, 804..."


In [ ]:

def jaccard(set1, set2):
    inters = set1.intersection(set2)
    union = set1.union(set2)
    return len(inters) / len(union) if union != 0 else 0.0

